In [1]:
from timeprocess import output_time
from genreprocess import output_genre
import json
import threading
from time import ctime,sleep
import time
import Queue
import requests
from collections import Counter
from flask import Flask

In [2]:
def worker_time(line,q):
    return_string=output_time(line)
    q.put(return_string)
def worker_genre(line,q):
    return_string=output_genre(line)
    q.put(return_string)
def worker_name(line,q):
    line = line.lower()
    start_name = time.clock()

    github_url = 'http://101.200.159.42:9001/vinci/naturelang/v2/metasearch?query='+line
    r = requests.post(github_url)
    try:
        content_list=r.json()['content']
        interface_singer_list = []
        for i in range(len(content_list)):
            interface_singer_list.append(content_list[i]['singer'])
        
        singer_counts = Counter(interface_singer_list)
        top_three = singer_counts.most_common(3)      
        synonym_singer = ''
        for word in top_three:
            if word[0].encode('utf8') in synonym:
                synonym_singer = synonym[word[0].encode('utf8')].replace('\n','')
            else:
                synonym_singer = 'None'

            if word[0].encode('utf8') in line:
                if word[0].encode('utf8') in all_singers:
                    singer = word[0]
                    break
            elif synonym_singer.encode('utf8') in line:
                if synonym_singer.encode('utf8') in all_singers:
                    singer = synonym_singer
                    break


        tmp={'artist':singer}

        q.put(tmp)
        end_name= time.clock()    
        print "read: %f s" % (end_name - start_name)
    except:
        tmp={'artist':'None'}
        q.put(tmp)


def worker_name_single(line,p):
    github_url='http://101.200.159.42:9001/vinci/naturelang/v2/getsemantic?query='+line
    r = requests.post(github_url)
    tmp_song='None'
    tmp_artist='None'
    try:
        tmp_artist=r.json()['content']['artist']
    except:
        pass
    try:
        tmp_song=r.json()['content']['song']
    except:
        pass
    
    if tmp_artist =='None' and tmp_song=='None':
        p.put(['None',1])
    else:
        tmp={
            'artist':tmp_artist,
            'song':tmp_song
        }
        p.put([tmp,0])

In [3]:
all_singers=[]
singer_file = open("/Users/fei_Daniel/Desktop/query理解/多线程/data/xiami_singer.dic","r")
lines = singer_file.readlines()
for line in lines:
    all_singers.append(line.strip("\n"))
singer_file.close()


synonym = {}
tup_in_synonym = [] 
synonym_file = open("/Users/fei_Daniel/Desktop/query理解/多线程/synonym.txt")
synonym_pair = synonym_file.readlines()
for line in synonym_pair:
    tup_in_synonym = line.split('\t')
    synonym[tup_in_synonym[0]] = tup_in_synonym[1]

In [7]:
def test_f(question):
        question=question.lower()
        q = Queue.Queue()
        p = Queue.Queue()

        
        threads = []
        t1 = threading.Thread(target=worker_time(question, q))
        threads.append(t1)
        t2 = threading.Thread(target=worker_genre(question, q))
        threads.append(t2)
        t3 = threading.Thread(target=worker_name(question ,q))
        threads.append(t3)
        t4 = threading.Thread(target=worker_name_single(question, p))
        threads.append(t4)



        for t in threads:
            t.setDaemon(True)
            t.start()
            t.join()


        test=p.get()


        if test[1]==1:

            result=[]
            while not q.empty():
                result.append(q.get())

            return json.dumps(result,ensure_ascii=False)
        elif test[1]==0:
            return json.dumps(test[0],ensure_ascii=False)

In [10]:
question='来个Rap'
print json.dumps(test_f(question),ensure_ascii=False)